### CREAMOS EL ARCHIVO SQL


In [16]:
with open("consulta.sql", "w") as file:
    file.write("SELECT * FROM Ejemplo;")

### CREAMOS NUESTRA LIBRERIA Lector_deConsulta

In [ ]:
from Lector_deConsulta import ConsultaDB

# Crear objeto lector
lector = ConsultaDB()

# Ejecutar consulta y obtener DataFrame
df = lector.ejecutar_consulta()

# Exportar: (elige el formato y la ruta)
# ❗Si no se especifica ruta ni formato, guarda como .parquet en la carpeta por defecto
lector.exportar(df)  # por defecto .parquet

# Ejemplo 1: exportar como CSV en la misma ruta
lector.exportar(df, nombre_archivo="datos_csv", formato="csv")

# Ejemplo 2: exportar como Excel en otra ruta
lector.exportar(df, nombre_archivo="datos_excel", formato="xlsx", ruta="/home/roma/Documentos/MisConsultas")


Conexión exitosa
Conexión cerrada.
Archivo guardado en: /home/roma/ENTREGAS/ENTREGAS/Carpeta_ejemplo/resultado.parquet
Archivo guardado en: /home/roma/ENTREGAS/ENTREGAS/Carpeta_ejemplo/datos_csv.csv
Archivo guardado en: /home/roma/Documentos/MisConsultas/datos_excel.xlsx


In [18]:
# Rutas de los archivos
ruta_parquet = "/home/roma/ENTREGAS/ENTREGAS/Carpeta_ejemplo/resultado.parquet"
ruta_csv = "/home/roma/ENTREGAS/ENTREGAS/Carpeta_ejemplo/datos_csv.csv"
ruta_excel = "/home/roma/Documentos/MisConsultas/datos_excel.xlsx"

# Leer los archivos
df_parquet = pd.read_parquet(ruta_parquet)
df_csv = pd.read_csv(ruta_csv)
df_excel = pd.read_excel(ruta_excel)

# Mostrar los tres DataFrames
print("📦 DataFrame desde .parquet:")
display(df_parquet)

print("📄 DataFrame desde .csv:")
display(df_csv)

print("📊 DataFrame desde .xlsx:")
display(df_excel)

📦 DataFrame desde .parquet:


,Casas,Empleado,Animales,Apartamente,Carros
0,1,2,3,4,5
1,2,3,4,5,6
2,3,4,5,6,7
3,8,9,0,1,2
4,3,4,5,6,7
5,8,9,0,1,2
6,3,4,5,6,7
7,8,9,0,1,2
8,3,4,5,6,7
9,8,9,0,1,2


📄 DataFrame desde .csv:


,Casas,Empleado,Animales,Apartamente,Carros
0,1,2,3,4,5
1,2,3,4,5,6
2,3,4,5,6,7
3,8,9,0,1,2
4,3,4,5,6,7
5,8,9,0,1,2
6,3,4,5,6,7
7,8,9,0,1,2
8,3,4,5,6,7
9,8,9,0,1,2


📊 DataFrame desde .xlsx:


,Casas,Empleado,Animales,Apartamente,Carros
0,1,2,3,4,5
1,2,3,4,5,6
2,3,4,5,6,7
3,8,9,0,1,2
4,3,4,5,6,7
5,8,9,0,1,2
6,3,4,5,6,7
7,8,9,0,1,2
8,3,4,5,6,7
9,8,9,0,1,2
